## Import Modules
---

In [1]:
import openpyxl
from openpyxl.utils.cell import range_boundaries
from pathlib import Path
import logging
import warnings
import time
import pandas as pd
import re

## Setup
---

In [2]:
# Use force=True if logging doesn't appear in the notebook otherwise
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s', force=True)
log = logging.getLogger(__name__)

# Suppress openpyxl warnings about data validation extensions if they occur
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [3]:
try:
    # Get the current working directory of the notebook kernel
    notebook_dir = Path.cwd()
    project_root = notebook_dir.parent # Go one level up from 'notebooks/' to the project root 'D:/'
    log.info(f"Notebook directory detected: {notebook_dir}")
    log.info(f"Assuming project root: {project_root}")
except Exception as e:
    log.error(f"Could not automatically determine project root. Please set 'project_root' manually. Error: {e}")
    # Fallback: Define project_root manually if needed (less portable)

INFO: Notebook directory detected: d:\Documents\phantom\phantom-canon\notebooks
INFO: Assuming project root: d:\Documents\phantom\phantom-canon


In [4]:
# Construct the path to the Excel file
excel_file_path = project_root / "data" / "knowledge_base.xlsx"

## Define Inspector
---

In [5]:
def inspect_excel_tables(file_path: Path) -> pd.DataFrame:
    """
    Inspects an Excel file (.xlsx) to extract all tables and their complete column structures.
    This function:
    1. First uses defined names to identify tables
    2. Then directly scans the workbook for Excel Tables to get all columns
    3. Falls back to extracting from defined names for tables not found as ListObjects
    """
    results = []
    if not file_path.is_file():
        print(f"ERROR: Excel file not found: {file_path}")
        log.error(f"Excel file not found: {file_path}")
        return pd.DataFrame(columns=['Sheet', 'Table', 'Column'])

    print(f"\nDEBUG: Starting inspection of Excel tables: {file_path.name}")
    log.info(f"Starting inspection of Excel tables: {file_path.name}")

    start_time = time.time()
    
    # First collect all table names from defined names
    table_names_from_refs = set()
    table_column_regex = re.compile(r"^(?P<table_name>[^[]+)\[(?P<column_name>[^\]]+)\]$", re.IGNORECASE)
    
    # Track table columns from defined names
    defined_name_columns = {}  # table_name -> set of column names

    try:
        print(f"DEBUG: Loading workbook in read-only mode for defined names... {time.strftime('%H:%M:%S')}")
        workbook_read_only = openpyxl.load_workbook(filename=file_path, read_only=True, data_only=True)
        
        # Extract table names from defined names
        if workbook_read_only.defined_names:
            defined_name_list = list(workbook_read_only.defined_names.values())
            print(f"DEBUG: Found {len(defined_name_list)} defined names. Scanning for table references...")
            
            for dn in defined_name_list:
                range_string = dn.value
                
                if not range_string or not isinstance(range_string, str):
                    continue
                    
                # Try to match Table[Column] pattern
                table_column_match = table_column_regex.match(range_string)
                if table_column_match:
                    table_name = table_column_match.group('table_name')
                    column_name = table_column_match.group('column_name')
                    
                    # Add to table names set
                    table_names_from_refs.add(table_name)
                    
                    # Add column to the table's column set
                    if table_name not in defined_name_columns:
                        defined_name_columns[table_name] = set()
                    defined_name_columns[table_name].add(column_name)
                    
            print(f"DEBUG: Extracted {len(table_names_from_refs)} unique table names from defined names: {table_names_from_refs}")
        
        # Close read-only workbook
        workbook_read_only.close()
        
        # Second pass: Try to get all actual Excel tables and their columns
        processed_tables = set()  # Keep track of tables we've already processed
        
        print(f"DEBUG: Loading workbook to access table objects... {time.strftime('%H:%M:%S')}")
        workbook = openpyxl.load_workbook(filename=file_path, data_only=True)
        
        # Process each sheet looking for tables
        sheets_with_tables = 0
        tables_found = 0
        
        for sheet_name in workbook.sheetnames:
            try:
                worksheet = workbook[sheet_name]
                
                # Check if worksheet has tables attribute and it's not empty
                if not hasattr(worksheet, 'tables') or not worksheet.tables:
                    continue
                    
                sheets_with_tables += 1
                print(f"DEBUG: Found {len(worksheet.tables)} tables in sheet '{sheet_name}'")
                
                # Process each table in the sheet
                for table_name in worksheet.tables:
                    tables_found += 1
                    print(f"DEBUG: Processing table '{table_name}' in sheet '{sheet_name}'")
                    
                    try:
                        # For newer versions of openpyxl, tables is a dict with table names as keys
                        # The actual table objects are in _tables
                        if hasattr(worksheet, '_tables') and table_name in worksheet._tables:
                            table_obj = worksheet._tables[table_name]
                            table_range = table_obj.ref
                            
                            # Parse the range to get row and column boundaries
                            min_col, min_row, max_col, max_row = range_boundaries(table_range)
                            
                            # Read the header row (first row of the table)
                            header_row = []
                            for col in range(min_col, max_col + 1):
                                cell = worksheet.cell(row=min_row, column=col)
                                if cell.value:
                                    header_row.append(str(cell.value).strip())
                            
                            print(f"DEBUG: Found {len(header_row)} columns in table '{table_name}': {header_row}")
                            
                            # Add each column to results
                            for column in header_row:
                                results.append({
                                    'Sheet': sheet_name,
                                    'Table': table_name,
                                    'Column': column
                                })
                            
                            # Mark this table as processed
                            processed_tables.add(table_name)
                            
                        else:
                            print(f"WARNING: Could not access table object for '{table_name}' in sheet '{sheet_name}'")
                    except Exception as table_err:
                        print(f"ERROR: Failed to process table '{table_name}' in sheet '{sheet_name}': {table_err}")
                        log.error(f"Error processing table '{table_name}': {table_err}")
            except Exception as sheet_err:
                print(f"ERROR: Failed to process sheet '{sheet_name}': {sheet_err}")
                log.error(f"Error processing sheet '{sheet_name}': {sheet_err}")
                continue
                
        # Third pass: Add columns from defined names for tables we didn't find as table objects
        for table_name in table_names_from_refs:
            if table_name not in processed_tables and table_name in defined_name_columns:
                print(f"DEBUG: Adding {len(defined_name_columns[table_name])} columns for table '{table_name}' from defined names")
                
                for column_name in defined_name_columns[table_name]:
                    results.append({
                        'Sheet': 'Unknown (From Defined Name)',
                        'Table': table_name,
                        'Column': column_name
                    })
                
                # Mark as processed
                processed_tables.add(table_name)
            
        print(f"DEBUG: Processed {len(processed_tables)} tables in total")
        workbook.close()
        
    except Exception as e:
        duration = time.time() - start_time
        print(f"ERROR: An exception occurred after {duration:.2f} seconds: {e}")
        log.error(f"Failed to process Excel file {file_path} after {duration:.2f} seconds: {e}", exc_info=True)
        import traceback
        print("\n--- TRACEBACK ---")
        traceback.print_exc()
        print("--- END TRACEBACK ---\n")
        return pd.DataFrame(columns=['Sheet', 'Table', 'Column'])

    total_duration = time.time() - start_time
    print(f"\nDEBUG: Table inspection finished. Duration: {total_duration:.2f} seconds.")
    log.info(f"Table inspection finished. Duration: {total_duration:.2f} seconds.")

    if not results:
        print("WARNING: No table structures found in the workbook.")
        log.warning("No table structures found in the workbook.")
        return pd.DataFrame(columns=['Sheet', 'Table', 'Column'])

    df_structure = pd.DataFrame(results)
    df_structure = df_structure.dropna(subset=['Column'])
    df_structure = df_structure.drop_duplicates().reset_index(drop=True)

    print(f"INFO: Found {len(df_structure)} columns across {df_structure['Table'].nunique()} tables in {df_structure['Sheet'].nunique()} sheets.")
    log.info(f"Inspection complete. Found {len(df_structure)} columns across {df_structure['Table'].nunique()} tables in {df_structure['Sheet'].nunique()} sheets.")
    
    return df_structure

## Execute
---

In [6]:
excel_structure_df = inspect_excel_tables(excel_file_path)

INFO: Starting inspection of Excel tables: knowledge_base.xlsx



DEBUG: Starting inspection of Excel tables: knowledge_base.xlsx
DEBUG: Loading workbook in read-only mode for defined names... 21:36:58
DEBUG: Found 23 defined names. Scanning for table references...
DEBUG: Extracted 11 unique table names from defined names: {'Table_Languages', 'Centuries_Table', 'Table_Books', 'Table_Score_0_5_Halves', 'Art_Movements_Table', 'Table_Art_Types', 'Table_Months', 'Table_Publishing_Houses', 'Table_People', 'Table_Book_Series', 'Table_Score_0_5_Fulls'}
DEBUG: Loading workbook to access table objects... 21:36:58


INFO: Table inspection finished. Duration: 95.27 seconds.
INFO: Inspection complete. Found 175 columns across 30 tables in 27 sheets.


DEBUG: Found 1 tables in sheet 'Occult_Library'
DEBUG: Processing table 'Table_Occult_Library' in sheet 'Occult_Library'
DEBUG: Found 13 columns in table 'Table_Occult_Library': ['Entry_Number', 'Category_01', 'Category_01_Number', 'Category_02', 'Category_02_Number', 'Title', 'Author', 'Edition/Translation', 'Edition_Language', 'Original_Language', 'Type', 'Description', 'Notes']
DEBUG: Found 1 tables in sheet 'People'
DEBUG: Processing table 'Table_People' in sheet 'People'
DEBUG: Found 21 columns in table 'Table_People': ['Hash_ID', 'Name', 'Surname', 'Real Name', 'Type', 'Gender', 'Nationality', 'Birth_Date_Day', 'Birth_Date_Month', 'Birth_Date_Year', 'Birth_Date_Year_Gregorian', 'Birth_Date_Year_IsRange', 'Death_Date_Day', 'Death_Date_Month', 'Death_Date_Year', 'Death_Date_Year_Gregorian', 'Death_Date_Year_IsRange', 'Complete Name (Name Surname)', 'Complete Name (Surname Name)', 'Duplicated_Entry', 'Has_Image [Y/N]']
DEBUG: Found 1 tables in sheet 'Calendars_Gregorian'
DEBUG: Proc

In [7]:
excel_structure_df

,Sheet,Table,Column
0,Occult_Library,Table_Occult_Library,Entry_Number
1,Occult_Library,Table_Occult_Library,Category_01
2,Occult_Library,Table_Occult_Library,Category_01_Number
3,Occult_Library,Table_Occult_Library,Category_02
4,Occult_Library,Table_Occult_Library,Category_02_Number
...,...,...,...
170,Countries_Continents,Table_Countries,Two_Letter_Country_Code
171,Countries_Continents,Table_Countries,Three_Letter_Country_Code
172,Countries_Continents,Table_Countries,Country_Number
173,Hash_ID_Acronyms,Table18,Field
